## Pandas optimizations

In [1]:
import pandas as pd
import gc

In [2]:
df = pd.read_csv("../ex04/fines.csv")
df

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2.0,3200.0,Ford,Focus,1989
1,E432XX77RUS,1.0,6500.0,Toyota,Camry,1995
2,7184TT36RUS,1.0,2100.0,Ford,Focus,1984
3,X582HE161RUS,2.0,2000.0,Ford,Focus,2015
4,92918M178RUS,1.0,5700.0,Ford,Focus,2014
...,...,...,...,...,...,...
925,X123YZ77RUS,1.0,5000.0,Honda,Civic,1995
926,Y456AB77RUS,2.0,3200.0,Nissan,Sentra,2005
927,Z789CD77RUS,1.0,4100.0,Kia,Rio,2010
928,A012EF77RUS,2.0,2800.0,Hyundai,Elantra,2015


### Iterations:

### For Loop + iloc + append

In [3]:
%%timeit
new_column = []
for i in range(len(df)):
    new_column.append(df.iloc[i]['Fines'] / df.iloc[i]['Refund'] * df.iloc[i]['Year'])

df['Calculated'] = new_column

106 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### iterrows()

In [4]:
%%timeit
new_column = []
for _, row in df.iterrows():
    new_column.append(row['Fines'] / row['Refund'] * row['Year'])

df['Calculated'] = new_column

34.8 ms ± 327 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### apply() and lambda

In [5]:
%%timeit
df['Calculated'] = df.apply(lambda row: row['Fines'] / row['Refund'] * row['Year'], axis=1)

7.07 ms ± 111 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Series Objects

In [6]:
%%timeit
df['Calculated'] = (df['Fines'] / df['Refund'] * df['Year'])  # eng tezi

178 μs ± 4.56 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### .values

In [7]:
%%timeit
fines = df['Fines'].values
refund = df['Refund'].values
year = df['Year'].values

df['Calculated'] = (fines / refund * year)

84.3 μs ± 2.11 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Indexing:

### Get a row

In [8]:
%%timeit
df.reset_index(inplace=True)
df.set_index('CarNumber', inplace=True)
car_row = df.loc['O136HO197RUS']

659 μs ± 42.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Re-set Index

In [9]:
df.reset_index(inplace=True)
df.set_index('CarNumber', inplace=True)
car_row = df.loc['O136HO197RUS']

In [10]:
car_row

,index,Refund,Fines,Make,Model,Year,Calculated
CarNumber,,,,,,,
O136HO197RUS,715,2.0,7800.0,Toyota,Corolla,1999,7796100.0
O136HO197RUS,902,1.0,300.0,Toyota,Corolla,1991,597300.0


### Downcasting

In [11]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to B345GH77RUS
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       930 non-null    int64  
 1   Refund      930 non-null    float64
 2   Fines       930 non-null    float64
 3   Make        930 non-null    object 
 4   Model       919 non-null    object 
 5   Year        930 non-null    int64  
 6   Calculated  930 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 221.5 KB


### Downcast from float64 to float32

In [12]:
# Float ustunlarini float64 dan float32 ga o‘zgartiramiz:
df_optimized = df.copy()

# Float ustunlarini downcast qilish
df_optimized['Fines'] = pd.to_numeric(df_optimized['Fines'], downcast='float')
df_optimized['Refund'] = pd.to_numeric(df_optimized['Refund'], downcast='float')

# Info ni qayta tekshiramiz
df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to B345GH77RUS
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       930 non-null    int64  
 1   Refund      930 non-null    float32
 2   Fines       930 non-null    float32
 3   Make        930 non-null    object 
 4   Model       919 non-null    object 
 5   Year        930 non-null    int64  
 6   Calculated  930 non-null    float64
dtypes: float32(2), float64(1), int64(2), object(2)
memory usage: 214.3 KB


### Downcast from int64 to smallest numerical dtype

In [13]:
# Butun sonlarni (integer) eng kichik turga o‘zgartiramiz:
# integer sonlarni eng kichik turga o‘zgartirish
df_optimized['Year'] = pd.to_numeric(df_optimized['Year'], downcast='integer')

# Info ni qayta tekshiramiz
df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 930 entries, Y163O8161RUS to B345GH77RUS
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       930 non-null    int64  
 1   Refund      930 non-null    float32
 2   Fines       930 non-null    float32
 3   Make        930 non-null    object 
 4   Model       919 non-null    object 
 5   Year        930 non-null    int16  
 6   Calculated  930 non-null    float64
dtypes: float32(2), float64(1), int16(1), int64(1), object(2)
memory usage: 208.8 KB


### Categories

In [14]:
# Matn ustunlarini category tipiga o'zgartirish, xotira tejash uchun
df_optimized.reset_index(inplace=True)
df_optimized['CarNumber'] = df_optimized['CarNumber'].astype('category')

df_optimized.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   CarNumber   930 non-null    category
 1   index       930 non-null    int64   
 2   Refund      930 non-null    float32 
 3   Fines       930 non-null    float32 
 4   Make        930 non-null    object  
 5   Model       919 non-null    object  
 6   Year        930 non-null    int16   
 7   Calculated  930 non-null    float64 
dtypes: category(1), float32(2), float64(1), int16(1), int64(1), object(2)
memory usage: 171.2 KB


###  Memory Clean

In [15]:
# Faqat boshlang'ich DataFrame'ni tozalaydi
del df
gc.collect()

0